In [ ]:
from mcts import MCTS
from generator import Generator
from utils import Utils
from backtrack import Backtrack

import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Generating puzzles

In [ ]:
gen = Generator()
ps = [gen.getPuzzle(i) for i in range(4, 15) for _ in range(300)]
ps = {pid: p for pid, p in enumerate(ps)}
print(f"{len(ps)} puzzles to solve")

Creating a list of dictionaries which we'll turn into a dataframe later

In [ ]:
data = []

Main loop of solving puzzles

In [ ]:
for pid in ps:
    print(f"{int(pid/len(ps) * 100)}% done")

    # MCTS, random
    mcts = MCTS(ps[pid])

    start = time.process_time()
    mcts.run()
    stop = time.process_time()
    
    t = stop - start
    vn = mcts.visited
    
    data.append({
        "pid":pid,
        "n":len(ps[pid].ps),
        "method":"mcts",
        "heuristic":"false",
        "visited_nodes":vn,
        "time":t
    })

    # MCTS, Rank
    mcts = MCTS(ps[pid], h = "R")

    start = time.process_time()
    mcts.run()
    stop = time.process_time()
    
    t = stop - start
    vn = mcts.visited

    data.append({
        "pid":pid,
        "n":len(ps[pid].ps),
        "method":"mcts",
        "heuristic":"true",
        "visited_nodes":vn,
        "time":t
    })

    # Backtrack, random
    bt = Backtrack(ps[pid])

    start = time.process_time()
    bt.run()
    stop = time.process_time()

    t = stop - start
    vn = bt.visited

    data.append({
        "pid":pid,
        "n":len(ps[pid].ps),
        "method":"bt",
        "heuristic":"false",
        "visited_nodes":vn,
        "time":t
    })

    # Backtrack, Rank
    bt = Backtrack(ps[pid], h = "R")

    start = time.process_time()
    bt.run()
    stop = time.process_time()

    t = stop - start
    vn = bt.visited

    data.append({
        "pid":pid,
        "n":len(ps[pid].ps),
        "method":"bt",
        "heuristic":"true",
        "visited_nodes":vn,
        "time":t
    })

print("DONE")

In [ ]:
data = pd.DataFrame(data)
data.to_csv("./data.csv")

means_vn = data.groupby(["method", "n"])["visited_nodes"].mean().reset_index()
means_t = data.groupby(["method", "n"])["time"].mean().reset_index()

In [ ]:
sns.set_theme(style="whitegrid")
sns.barplot(data=means_vn, x = "n", hue = "method", y = "visited_nodes")

In [ ]:
p = gen.getPuzzle(10)
vn = []

for _ in range(10):
    mcts = MCTS(p)
    mcts.run()
    vn.append(mcts.visited)

print(vn)